In [1]:
import classes
import caffemodel2pytorch as caffe

# modified visualpriors library
from transforms import VisualPriorRepresentation, VisualPriorPredictedLabel
from taskonomy_network import TaskonomyEncoder, TaskonomyDecoder

import os
import collections
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import torchsummary

from PIL import Image
from scipy.stats import pearsonr
from scipy.io import savemat

import torch.utils.model_zoo # required to load nets
import torchvision.transforms.functional as TF
from torch import nn
import torchvision.models
from torchvision.models.feature_extraction import get_graph_node_names, create_feature_extractor

#%load_ext autoreload
#%autoreload 2

# Run analysis for vgg16_places

To confirm correcteness of implementation

In [32]:
class vgg16_places_model(nn.Module):
    def __init__(self):
        super(vgg16_places_model, self).__init__()
        
        self.conv_layers = nn.Sequential(
            collections.OrderedDict(
                [
                    ('conv1_1', nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, stride=1, padding=1)),
                    ('ReLU1_1', nn.ReLU()),
                    ('conv1_2', nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1, padding=1)),
                    ('ReLU1_2',nn.ReLU()),
                    ('MaxPool1', nn.MaxPool2d(kernel_size=2, stride=2,padding=0)),
                    ('conv2_1',nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1)),
                    ('ReLU2_1',nn.ReLU()),
                    ('conv2_2',nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, stride=1, padding=1)),
                    ('ReLU2_2',nn.ReLU()),
                    ('MaxPool2', nn.MaxPool2d(kernel_size=2, stride=2,padding=0)),
                    ('conv3_1', nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, stride=1, padding=1)),
                    ('ReLU3_1',nn.ReLU()),
                    ('conv3_2', nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, stride=1, padding=1)),
                    ('ReLU3_2',nn.ReLU()),
                    ('conv3_3', nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, stride=1, padding=1)),
                    ('ReLU3_3',nn.ReLU()),
                    ('MaxPool3', nn.MaxPool2d(kernel_size=2, stride=2,padding=0)),
                    ('conv4_1', nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, stride=1, padding=1)),
                    ('ReLU4_1',nn.ReLU()),
                    ('conv4_2', nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, stride=1, padding=1)),
                    ('ReLU4_2',nn.ReLU()),
                    ('conv4_3', nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, stride=1, padding=1)),
                    ('ReLU4_3',nn.ReLU()),
                    ('MaxPool4', nn.MaxPool2d(kernel_size=2, stride=2,padding=0)),
                    ('conv5_1', nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, stride=1, padding=1)),
                    ('ReLU5_1',nn.ReLU()),
                    ('conv5_2', nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, stride=1, padding=1)),
                    ('ReLU5_2',nn.ReLU()),
                    ('conv5_3', nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, stride=1, padding=1)),
                    ('ReLU5_3',nn.ReLU()),
                    ('MaxPool5', nn.MaxPool2d(kernel_size=2, stride=2,padding=0))
            ]))
        self.fullyconnected_layers = nn.Sequential(
            collections.OrderedDict(
                [
                    ('fc6', nn.Linear(in_features=25088, out_features=4096)),
                    ('ReLU6',nn.ReLU()),
                    ('fc7', nn.Linear(in_features=4096, out_features=4096)),
                    ('ReLU7',nn.ReLU()),
                    ('fc8a', nn.Linear(in_features=4096, out_features=365)),
                    ('Softmax8a', nn.Softmax(dim=-1))
            ]))
    

    def forward(self, x):
        x = self.conv_layers(x)
        x = self.fullyconnected_layers(x)
        return x



In [33]:
test = vgg16_places_model()
test

vgg16_places_model(
  (conv_layers): Sequential(
    (conv1_1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (ReLU1_1): ReLU()
    (conv1_2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (ReLU1_2): ReLU()
    (MaxPool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (conv2_1): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (ReLU2_1): ReLU()
    (conv2_2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (ReLU2_2): ReLU()
    (MaxPool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (conv3_1): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (ReLU3_1): ReLU()
    (conv3_2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (ReLU3_2): ReLU()
    (conv3_3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (ReLU3_3): ReLU()
    (MaxPool3): MaxPool2d(kernel_size=2,

In [34]:
_, eval_nodes = get_graph_node_names(test)
for node in eval_nodes:
    print(node, end='   ')

x   conv_layers.conv1   conv_layers.ReLU1   conv_layers.conv1_1   conv_layers.ReLU1_1   conv_layers.MaxPool1   conv_layers.conv2   conv_layers.ReLU2   conv_layers.conv2_1   conv_layers.ReLU2_1   conv_layers.MaxPool2   conv_layers.conv3   conv_layers.ReLU3   conv_layers.conv3_1   conv_layers.ReLU3_1   conv_layers.conv3_2   conv_layers.ReLU3_2   conv_layers.MaxPool3   conv_layers.conv4   conv_layers.ReLU4   conv_layers.conv4_1   conv_layers.ReLU4_1   conv_layers.conv4_2   conv_layers.ReLU4_2   conv_layers.MaxPool4   conv_layers.conv5   conv_layers.ReLU5   conv_layers.conv5_1   conv_layers.ReLU5_1   conv_layers.conv5_2   conv_layers.ReLU5_2   conv_layers.MaxPool5   fullyconnected_layers.fc6   fullyconnected_layers.ReLU6   fullyconnected_layers.fc7   fullyconnected_layers.ReLU7   fullyconnected_layers.fc8a   fullyconnected_layers.Softmax8a   

In [3]:
vgg16_places = nn.Sequential(
    collections.OrderedDict(
        [
            ('conv1_1', nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, stride=1, padding=1)),
            ('ReLU1_1', nn.ReLU()),
            ('conv1_2', nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1, padding=1)),
            ('ReLU1_2',nn.ReLU()),
            ('MaxPool1', nn.MaxPool2d(kernel_size=2, stride=2,padding=0)),
            ('conv2_1',nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1)),
            ('ReLU2_1',nn.ReLU()),
            ('conv2_2',nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, stride=1, padding=1)),
            ('ReLU2_2',nn.ReLU()),
            ('MaxPool2', nn.MaxPool2d(kernel_size=2, stride=2,padding=0)),
            ('conv3_1', nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, stride=1, padding=1)),
            ('ReLU3_1',nn.ReLU()),
            ('conv3_2', nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, stride=1, padding=1)),
            ('ReLU3_2',nn.ReLU()),
            ('conv3_3', nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, stride=1, padding=1)),
            ('ReLU3_3',nn.ReLU()),
            ('MaxPool3', nn.MaxPool2d(kernel_size=2, stride=2,padding=0)),
            ('conv4_1', nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, stride=1, padding=1)),
            ('ReLU4_1',nn.ReLU()),
            ('conv4_2', nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, stride=1, padding=1)),
            ('ReLU4_2',nn.ReLU()),
            ('conv4_3', nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, stride=1, padding=1)),
            ('ReLU4_3',nn.ReLU()),
            ('MaxPool4', nn.MaxPool2d(kernel_size=2, stride=2,padding=0)),
            ('conv5_1', nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, stride=1, padding=1)),
            ('ReLU5_1',nn.ReLU()),
            ('conv5_2', nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, stride=1, padding=1)),
            ('ReLU5_2',nn.ReLU()),
            ('conv5_3', nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, stride=1, padding=1)),
            ('ReLU5_3',nn.ReLU()),
            ('MaxPool5', nn.MaxPool2d(kernel_size=2, stride=2,padding=0)),
            #('Flatten6', nn.Flatten()),
            ('fc6', nn.Linear(in_features=25088, out_features=4096)),
            ('ReLU6',nn.ReLU()),
            ('fc7', nn.Linear(in_features=4096, out_features=4096)),
            ('ReLU7',nn.ReLU()),
            ('fc8a', nn.Linear(in_features=4096, out_features=365)),
            ('Softmax8a', nn.Softmax(dim=-1))
        ]
    )
)
torchsummary.summary(vgg16_places, input_size=(3,224,224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 224, 224]           1,792
              ReLU-2         [-1, 64, 224, 224]               0
            Conv2d-3         [-1, 64, 224, 224]          36,928
              ReLU-4         [-1, 64, 224, 224]               0
         MaxPool2d-5         [-1, 64, 112, 112]               0
            Conv2d-6        [-1, 128, 112, 112]          73,856
              ReLU-7        [-1, 128, 112, 112]               0
            Conv2d-8        [-1, 128, 112, 112]         147,584
              ReLU-9        [-1, 128, 112, 112]               0
        MaxPool2d-10          [-1, 128, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]         295,168
             ReLU-12          [-1, 256, 56, 56]               0
           Conv2d-13          [-1, 256, 56, 56]         590,080
             ReLU-14          [-1, 256,

In [28]:
vgg16_places

Sequential(
  (conv1_1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (ReLU1_1): ReLU()
  (conv1_2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (ReLU1_2): ReLU()
  (MaxPool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2_1): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (ReLU2_1): ReLU()
  (conv2_2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (ReLU2_2): ReLU()
  (MaxPool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3_1): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (ReLU3_1): ReLU()
  (conv3_2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (ReLU3_2): ReLU()
  (conv3_3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (ReLU3_3): ReLU()
  (MaxPool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv4_1): Conv2d(

In [8]:
_, eval_nodes = get_graph_node_names(vgg16_places)
for node in eval_nodes:
    print(node, end='   ')

input   conv1   ReLU1   conv1_1   ReLU1_1   MaxPool1   conv2   ReLU2   conv2_1   ReLU2_1   MaxPool2   conv3   ReLU3   conv3_1   ReLU3_1   conv3_2   ReLU3_2   MaxPool3   conv4   ReLU4   conv4_1   ReLU4_1   conv4_2   ReLU4_2   MaxPool4   conv5   ReLU5   conv5_1   ReLU5_1   conv5_2   ReLU5_2   MaxPool5   Flatten6   fc6   ReLU6   fc7   ReLU7   fc8a   Softmax8a   

In [10]:
return_nodes = { node:node for node in eval_nodes if "conv" in node or 'fc' in node}
return_nodes

In [12]:
# maybe implement automatic extraction of all conv and fc layers, given just the style in specific network after inspection
vgg16_places_fe = create_feature_extractor(vgg16_places, return_nodes=return_nodes)


In [15]:
places_images = classes.ImageDataset('./data/stimuli_places1', beauty_ratings_path='./behavior/ratings_study1.csv')

In [16]:
integration_calc = classes.IntegrationCalculator(vgg16_places_fe, return_nodes)

In [17]:
classes.calculate_dataset_integration(places_images, integration_calc)

RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x204800 and 25088x4096)

## Export activations to matlab for plotting

# Analysis for one taskonomy model

In [2]:
task = 'autoencoding'

VisualPriorRepresentation._load_unloaded_nets([task]) # preload nets
net = VisualPriorRepresentation.feature_task_to_net[task] # loads encoder only for representations
torchsummary.summary(net, (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]           4,096
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]          16,384
      BatchNorm2d-12          [-1, 256, 56, 56]             512
           Conv2d-13          [-1, 256, 56, 56]          16,384
      BatchNorm2d-14          [-1, 256,

In [3]:
_, eval_nodes = get_graph_node_names(net)
for node in eval_nodes:
    print(node, end='   ')

x   pad   conv1   bn1   relu   maxpool   layer1.0.conv1   layer1.0.bn1   layer1.0.relu   layer1.0.conv2   layer1.0.bn2   layer1.0.relu_1   layer1.0.conv3   layer1.0.bn3   layer1.0.downsample.0   layer1.0.downsample.1   layer1.0.add   layer1.0.relu_2   layer1.1.conv1   layer1.1.bn1   layer1.1.relu   layer1.1.conv2   layer1.1.bn2   layer1.1.relu_1   layer1.1.conv3   layer1.1.bn3   layer1.1.add   layer1.1.relu_2   layer1.2.conv1   layer1.2.bn1   layer1.2.relu   layer1.2.conv2   layer1.2.bn2   layer1.2.relu_1   layer1.2.conv3   layer1.2.bn3   layer1.2.downsample.0   layer1.2.add   layer1.2.relu_2   layer2.0.conv1   layer2.0.bn1   layer2.0.relu   layer2.0.conv2   layer2.0.bn2   layer2.0.relu_1   layer2.0.conv3   layer2.0.bn3   layer2.0.downsample.0   layer2.0.downsample.1   layer2.0.add   layer2.0.relu_2   layer2.1.conv1   layer2.1.bn1   layer2.1.relu   layer2.1.conv2   layer2.1.bn2   layer2.1.relu_1   layer2.1.conv3   layer2.1.bn3   layer2.1.add   layer2.1.relu_2   layer2.2.conv1   layer2.

In [4]:
return_nodes = { node:node for node in eval_nodes if "conv" in node or 'fc' in node}
return_nodes

{'conv1': 'conv1',
 'layer1.0.conv1': 'layer1.0.conv1',
 'layer1.0.conv2': 'layer1.0.conv2',
 'layer1.0.conv3': 'layer1.0.conv3',
 'layer1.1.conv1': 'layer1.1.conv1',
 'layer1.1.conv2': 'layer1.1.conv2',
 'layer1.1.conv3': 'layer1.1.conv3',
 'layer1.2.conv1': 'layer1.2.conv1',
 'layer1.2.conv2': 'layer1.2.conv2',
 'layer1.2.conv3': 'layer1.2.conv3',
 'layer2.0.conv1': 'layer2.0.conv1',
 'layer2.0.conv2': 'layer2.0.conv2',
 'layer2.0.conv3': 'layer2.0.conv3',
 'layer2.1.conv1': 'layer2.1.conv1',
 'layer2.1.conv2': 'layer2.1.conv2',
 'layer2.1.conv3': 'layer2.1.conv3',
 'layer2.2.conv1': 'layer2.2.conv1',
 'layer2.2.conv2': 'layer2.2.conv2',
 'layer2.2.conv3': 'layer2.2.conv3',
 'layer2.3.conv1': 'layer2.3.conv1',
 'layer2.3.conv2': 'layer2.3.conv2',
 'layer2.3.conv3': 'layer2.3.conv3',
 'layer3.0.conv1': 'layer3.0.conv1',
 'layer3.0.conv2': 'layer3.0.conv2',
 'layer3.0.conv3': 'layer3.0.conv3',
 'layer3.1.conv1': 'layer3.1.conv1',
 'layer3.1.conv2': 'layer3.1.conv2',
 'layer3.1.conv3': 

In [5]:
net_tweaked = create_feature_extractor(net, return_nodes=return_nodes)
net_tweaked

TaskonomyEncoder(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (layer1): Module(
    (0): Module(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (downsample): Module(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
   

In [6]:
places_images = classes.ImageDataset('./data/stimuli_places1', beauty_ratings_path='./behavior/ratings_study1.csv')

In [7]:
integration_calc = classes.IntegrationCalculator(net_tweaked, return_nodes)

In [8]:
result = classes.calculate_dataset_integration(places_images, integration_calc)

250it [14:17,  3.43s/it]


In [9]:
result

,conv1,layer1.0.conv1,layer1.0.conv2,layer1.0.conv3,layer1.1.conv1,layer1.1.conv2,layer1.1.conv3,layer1.2.conv1,layer1.2.conv2,layer1.2.conv3,...,layer3.5.conv3,layer4.0.conv1,layer4.0.conv2,layer4.0.conv3,layer4.1.conv1,layer4.1.conv2,layer4.1.conv3,layer4.2.conv1,layer4.2.conv2,layer4.2.conv3
0,-0.901755,-0.927244,-0.906128,-0.892599,-0.924830,-0.857169,-0.839657,-0.941186,-0.896023,-0.852555,...,-0.859965,-0.995464,-0.858975,-0.725659,-0.895166,-0.546067,-0.366636,-0.620608,-0.348664,-0.371284
1,-0.977258,-0.979798,-0.977366,-0.970708,-0.973443,-0.941702,-0.918240,-0.974717,-0.968564,-0.947710,...,-0.938060,-0.996411,-0.912282,-0.805249,-0.957089,-0.706170,-0.470574,-0.755524,-0.356730,-0.136613
2,-0.985005,-0.988065,-0.982306,-0.978973,-0.980807,-0.959042,-0.937909,-0.980815,-0.977924,-0.965159,...,-0.932084,-0.996749,-0.916451,-0.840916,-0.964348,-0.723489,-0.497462,-0.776550,-0.360851,-0.148527
3,-0.969972,-0.971745,-0.968256,-0.959294,-0.971802,-0.943989,-0.924279,-0.974445,-0.967046,-0.948057,...,-0.903482,-0.997028,-0.915133,-0.862330,-0.956345,-0.715045,-0.582552,-0.792562,-0.507871,-0.420497
4,-0.982985,-0.985039,-0.980419,-0.975774,-0.981731,-0.966215,-0.950199,-0.980384,-0.976013,-0.963792,...,-0.902900,-0.996946,-0.916627,-0.884209,-0.963456,-0.755956,-0.626542,-0.805912,-0.571683,-0.499548
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,-0.967946,-0.978558,-0.971832,-0.965229,-0.966798,-0.933618,-0.914758,-0.967229,-0.955363,-0.933746,...,-0.926590,-0.995430,-0.902838,-0.806128,-0.947222,-0.727485,-0.504238,-0.768200,-0.392083,-0.184156
246,-0.971098,-0.974567,-0.971881,-0.963860,-0.971524,-0.946957,-0.928143,-0.972739,-0.964131,-0.943081,...,-0.925275,-0.996563,-0.914013,-0.855937,-0.954991,-0.736623,-0.608247,-0.798507,-0.543984,-0.475614
247,-0.970213,-0.975484,-0.968955,-0.964136,-0.970212,-0.949807,-0.937206,-0.966894,-0.957912,-0.952441,...,-0.924776,-0.993317,-0.912605,-0.901262,-0.942384,-0.802596,-0.611354,-0.816946,-0.436379,-0.208226
248,-0.991683,-0.993783,-0.991791,-0.989363,-0.987538,-0.974684,-0.956181,-0.986093,-0.987357,-0.976262,...,-0.924651,-0.997633,-0.923652,-0.856157,-0.972104,-0.729208,-0.493144,-0.781738,-0.362910,-0.120459


# Analysis for all taskonomy models

In [33]:
def run_taskonomy_encoder_integration_analysis(task, places_images):
    VisualPriorRepresentation._load_unloaded_nets([task])
    net = VisualPriorRepresentation.feature_task_to_net[task]

    _, eval_nodes = get_graph_node_names(net)
    return_nodes = { node:node for node in eval_nodes if "conv" in node or 'fc' in node}
    net_tweaked = create_feature_extractor(net, return_nodes=return_nodes)

    integration_calc = classes.IntegrationCalculator(net_tweaked, return_nodes)
    return classes.calculate_dataset_integration(places_images, integration_calc)

In [21]:
tasks = ('autoencoding','depth_euclidean','jigsaw','reshading',
         'colorization', 'edge_occlusion','keypoints2d','room_layout',
         'curvature','edge_texture','keypoints3d','segment_unsup2d',
         'class_object','egomotion','nonfixated_pose','segment_unsup25d',
         'class_scene','fixated_pose','normal','segment_semantic',
         'denoising','inpainting','point_matching','vanishing_point')

places_images = classes.ImageDataset('./data/stimuli_places1', beauty_ratings_path='./behavior/ratings_study1.csv')

mdict = {}
for task in tasks:
    task_result = run_taskonomy_encoder_integration_analysis(task, places_images)
    task_result.to_csv(os.path.join('results', task, task + '.csv'))
    #mdict[task] = run_taskonomy_encoder_integration_analysis(task, places_images)



In [ ]:

#savemat("taskonomy_integration")